
# Homework 2. LDA for classification
Jorge Alejandro Díaz Sánchez


The description of this task in PDF can be downloaded at the following link: Homework description file

Links to an external site.

In this exercise, the dataset that will be used is “Heart_Disease”. This database contains 76 attributes, but all published experiments refer to using a subset of 14 of them.  In particular, the Cleveland database is the only one that has been used by ML researchers to date.  The "goal" field refers to the presence of heart disease in the patient. It is integer valued from 0 (no presence) to 4. Experiments with the Cleveland database have concentrated on simply attempting to distinguish presence (values 1,2,3,4) from absence (value 0). The dataset can be obtained in the repository “UC Irvine Machine Learning Repository” (https://archive.ics.uci.edu/dataset/45/heart+disease
Links to an external site. ). For this exercise, you can use any of the files namely “processed.xxxxx.data”. You should consider that there are some differences between the classes in some of those files.

In [1]:
random_seed = 0

In [2]:
import sys
sys.path.append(r"~/Documents/GitHub/PredictiveModelling/")
import ax_utilities as utl
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv("Data/heart_disease/processed.cleveland.data", header=None, names=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num'], na_values=["?"])
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


Attribute Information:
   -- Only 14 used
      -- 1. #3  (age)
      -- 2. #4  (sex)      
      -- 3. #9  (cp)        
      -- 4. #10 (trestbps)  
      -- 5. #12 (chol)      
      -- 6. #16 (fbs)       
      -- 7. #19 (restecg)   
      -- 8. #32 (thalach)   
      -- 9. #38 (exang)     
      -- 10. #40 (oldpeak)   
      -- 11. #41 (slope)     
      -- 12. #44 (ca)        
      -- 13. #51 (thal)      
      -- 14. #58 (num)       (the predicted attribute)

In [4]:
utl.data_quality_report(df)

,0,Type,Missing_values,Present_values,Unique_values,Min_value,Max_value
age,age,float64,0,303,41,29.0,77.0
sex,sex,float64,0,303,2,0.0,1.0
cp,cp,float64,0,303,4,1.0,4.0
trestbps,trestbps,float64,0,303,50,94.0,200.0
chol,chol,float64,0,303,152,126.0,564.0
fbs,fbs,float64,0,303,2,0.0,1.0
restecg,restecg,float64,0,303,3,0.0,2.0
thalach,thalach,float64,0,303,91,71.0,202.0
exang,exang,float64,0,303,2,0.0,1.0
oldpeak,oldpeak,float64,0,303,40,0.0,6.2


There are some non numeric  characters "?" in variables "ca" and "thal", we are replacing them with np.nan, to drop them.

In [5]:
utl.data_quality_report(df)

,0,Type,Missing_values,Present_values,Unique_values,Min_value,Max_value
age,age,float64,0,303,41,29.0,77.0
sex,sex,float64,0,303,2,0.0,1.0
cp,cp,float64,0,303,4,1.0,4.0
trestbps,trestbps,float64,0,303,50,94.0,200.0
chol,chol,float64,0,303,152,126.0,564.0
fbs,fbs,float64,0,303,2,0.0,1.0
restecg,restecg,float64,0,303,3,0.0,2.0
thalach,thalach,float64,0,303,91,71.0,202.0
exang,exang,float64,0,303,2,0.0,1.0
oldpeak,oldpeak,float64,0,303,40,0.0,6.2


In [6]:
df.isna().sum().sum() / df.shape[0]

0.019801980198019802

In [7]:
df.dropna(inplace=True)

In [8]:
inputs = df.drop(columns="num")
inputs.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'],
      dtype='object')

In [9]:
target = df["num"]
target.value_counts()

0    160
1     54
2     35
3     35
4     13
Name: num, dtype: int64

In [10]:
# we are only interested if there's a hearth disease (1-4) or not (0), so we are grouping 1-4 into just 1
target.iloc[target != 0] = 1

/var/folders/51/l057c8xx2md597vhk47n57rm0000gn/T/ipykernel_74840/2469113571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target.iloc[target != 0] = 1


In [11]:
target.value_counts()

0    160
1    137
Name: num, dtype: int64

## Test and Train subsets

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=.30,random_state=random_seed)

## Useful Utility

In [14]:
from sklearn.metrics import (precision_score, accuracy_score, recall_score)

def get_model_metrics(actual, predicted):
    print("Accuracy: %0.2f"%(accuracy_score(actual, predicted)))
    print("Recall:  %0.2f"%recall_score(actual, predicted))
    print("Precision:  %0.2f"%precision_score(actual, predicted))

## Base Logistic Regression Model
Train a logistic regression model to estimate the feature “target” using all the pixels in the
image as input variables. Get the accuracy values or another metric (precision, recall, …) to
evaluate the model's performance in training and testing.

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
base_logistic_model = LogisticRegression()
base_logistic_model.fit(X_train, y_train)
y_predicted_train = base_logistic_model.predict(X_train)
y_predicted_test = base_logistic_model.predict(X_test)

/Users/adiaz/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
get_model_metrics(y_train, y_predicted_train)

Accuracy: 0.87
Recall:  0.80
Precision:  0.90


In [18]:
get_model_metrics(y_test, y_predicted_test)

Accuracy: 0.84
Recall:  0.76
Precision:  0.89


In [19]:
base_logistic_model.coef_.T

array([[-0.04244454],
       [ 0.60012529],
       [ 0.40485957],
       [ 0.02793079],
       [ 0.00226584],
       [-0.35816886],
       [ 0.09602762],
       [-0.04581619],
       [ 0.97076849],
       [ 0.34360938],
       [ 0.18096636],
       [ 0.9353186 ],
       [ 0.36108888]])

In [20]:
base_logistic_model.intercept_

array([-0.21725961])

## PCA + Logistic Regression
Considering the data set obtained in point 1, perform the dimension reduction using the
PCA method. With the variables resulting from the reduction process, train a new logistic
regression model and calculate the same metrics used in point 2.

In [21]:
# For this case, let's first transform the data using Principal Component Analysis
from sklearn.decomposition import PCA

In [22]:
pca_transformer = PCA()
X_train_pca = pca_transformer.fit_transform(X_train)
pca_transformer.explained_variance_ratio_

array([7.38612447e-01, 1.52629730e-01, 8.95568493e-02, 1.69537319e-02,
       1.06536070e-03, 3.35519911e-04, 2.57636474e-04, 2.17144156e-04,
       1.81549743e-04, 6.56415909e-05, 4.83608792e-05, 4.09784687e-05,
       3.50492726e-05])

In [23]:
from numpy import cumsum

cumsum(pca_transformer.explained_variance_ratio_)

array([0.73861245, 0.89124218, 0.98079903, 0.99775276, 0.99881812,
       0.99915364, 0.99941128, 0.99962842, 0.99980997, 0.99987561,
       0.99992397, 0.99996495, 1.        ])

In [24]:
pca_transformer.components_

array([[ 4.00814935e-02, -1.65916732e-03, -1.70475069e-04,
         4.82842284e-02,  9.97906419e-01,  1.45373320e-04,
         3.03170238e-03, -1.47056889e-02,  2.43064909e-04,
         2.78813738e-03,  2.65094973e-05,  2.88293574e-03,
         6.99576796e-04],
       [ 1.98494252e-01,  6.43654740e-06,  1.39238912e-02,
         1.25350665e-01, -2.84479198e-02,  2.91389291e-04,
         2.45440935e-03, -9.70982539e-01,  8.18167950e-03,
         1.52854661e-02,  8.88276449e-03,  1.20334373e-02,
         2.33793051e-02],
       [ 8.53162831e-02, -1.40321859e-03, -7.53510160e-03,
         9.84260114e-01, -4.89466411e-02,  3.71275519e-03,
         7.73566715e-03,  1.46112312e-01, -1.11459085e-03,
         5.08162646e-03,  1.67901019e-03,  9.51920161e-04,
         6.73909441e-03],
       [-9.74909179e-01,  7.28838236e-03,  1.03709943e-02,
         1.13621400e-01,  3.09968058e-02, -6.87043967e-03,
         2.91016189e-03, -1.85405675e-01,  8.16176368e-03,
         6.77057807e-03,  5.57035591e

In [25]:
import pickle

# open a file to save PCA Matrix
file = open('pca_matrix.pickle', 'wb')

# dump information to that file
pickle.dump(pca_transformer.components_, file)

# close the file
file.close()

In [50]:
X_train_pca = X_train_pca[:,:3]
X_test_pca = pca_transformer.transform(X_test)[:,:3]

In [51]:
X_train_pca.shape

(207, 3)

In [52]:
X_test_pca.shape

(90, 3)

In [53]:
pca_logistic_model = LogisticRegression()
pca_logistic_model.fit(X_train_pca, y_train)
y_predicted_train = pca_logistic_model.predict(X_train_pca)
y_predicted_test = pca_logistic_model.predict(X_test_pca)

In [54]:
get_model_metrics(y_train, y_predicted_train)

Accuracy: 0.74
Recall:  0.68
Precision:  0.73


In [55]:
get_model_metrics(y_test, y_predicted_test)

Accuracy: 0.66
Recall:  0.60
Precision:  0.64


In [65]:
pca_logistic_model.coef_.T

array([[0.00473673],
       [0.05054616],
       [0.01743115]])

In [68]:
pca_transformer.components_

array([[ 4.00814935e-02, -1.65916732e-03, -1.70475069e-04,
         4.82842284e-02,  9.97906419e-01,  1.45373320e-04,
         3.03170238e-03, -1.47056889e-02,  2.43064909e-04,
         2.78813738e-03,  2.65094973e-05,  2.88293574e-03,
         6.99576796e-04],
       [ 1.98494252e-01,  6.43654740e-06,  1.39238912e-02,
         1.25350665e-01, -2.84479198e-02,  2.91389291e-04,
         2.45440935e-03, -9.70982539e-01,  8.18167950e-03,
         1.52854661e-02,  8.88276449e-03,  1.20334373e-02,
         2.33793051e-02],
       [ 8.53162831e-02, -1.40321859e-03, -7.53510160e-03,
         9.84260114e-01, -4.89466411e-02,  3.71275519e-03,
         7.73566715e-03,  1.46112312e-01, -1.11459085e-03,
         5.08162646e-03,  1.67901019e-03,  9.51920161e-04,
         6.73909441e-03],
       [-9.74909179e-01,  7.28838236e-03,  1.03709943e-02,
         1.13621400e-01,  3.09968058e-02, -6.87043967e-03,
         2.91016189e-03, -1.85405675e-01,  8.16176368e-03,
         6.77057807e-03,  5.57035591e

In [57]:
pca_logistic_model.intercept_

array([-0.17380058])

## LDA Transform + Logistic Regression
Considering the data set used in point 1 again, perform a variable reduction by LDA
transformation. With the variables resulting from the transformation process, train a new
logistic regression wich must be evaluated with the same metrics as the previous models.

In [31]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [72]:
lda_transformer = LinearDiscriminantAnalysis()
X_train_lda = lda_transformer.fit_transform(X_train, y_train)


In [33]:
X_train_lda

array([[ 2.52265242],
       [ 2.07995297],
       [ 1.48892118],
       [ 2.10799989],
       [-1.5773806 ],
       [-1.49885784],
       [ 0.68265666],
       [-1.52192452],
       [ 1.64673239],
       [ 0.5887162 ],
       [ 1.13967296],
       [-0.36266394],
       [-0.53516217],
       [ 0.52276138],
       [ 1.80027948],
       [-2.20227122],
       [ 0.76543376],
       [ 0.29567724],
       [ 1.59944529],
       [-0.04451519],
       [ 0.83091644],
       [ 2.28902616],
       [ 2.5578495 ],
       [ 0.4428376 ],
       [-2.25588691],
       [ 1.31255251],
       [-2.24673738],
       [-0.68286823],
       [ 0.39523628],
       [-0.7735628 ],
       [-1.98082715],
       [ 2.40183388],
       [ 0.114578  ],
       [-2.04504752],
       [ 1.18488461],
       [ 2.68999539],
       [-2.09844424],
       [ 1.45530649],
       [-0.63121264],
       [-0.24355045],
       [ 0.21467643],
       [-1.55062299],
       [-0.20226969],
       [ 0.72435867],
       [ 1.75151035],
       [ 2

In [75]:
X_test_lda = lda_transformer.transform(X_test)
X_test_lda.shape

(90, 1)

In [69]:
lda_transformer.coef_.T

array([[ 0.00775993],
       [ 1.07450737],
       [ 0.79059315],
       [ 0.03740164],
       [ 0.00312264],
       [-0.76850344],
       [ 0.14315173],
       [-0.02352133],
       [ 1.56180146],
       [ 0.35808239],
       [ 0.48666282],
       [ 0.9030819 ],
       [ 0.49112669]])

In [70]:
lda_transformer.explained_variance_ratio_

array([1.])

In [35]:
# open a file to save LDA coefficients
file = open('lda_coef.pickle', 'wb')

# dump information to that file
pickle.dump(lda_transformer.coef_, file)

# close the file
file.close()

In [76]:
lda_logistic_model = LogisticRegression()
lda_logistic_model.fit(X_train_lda, y_train)
y_predicted_train = lda_logistic_model.predict(X_train_lda)
y_predicted_test = lda_logistic_model.predict(X_test_lda)

In [77]:
get_model_metrics(y_train, y_predicted_train)

Accuracy: 0.86
Recall:  0.83
Precision:  0.86


In [78]:
get_model_metrics(y_test, y_predicted_test)

Accuracy: 0.84
Recall:  0.76
Precision:  0.89


In [79]:
lda_logistic_model.coef_

array([[1.77827883]])

In [80]:
lda_logistic_model.intercept_

array([-0.14278611])

## LDA MODEL
Considering the data set used in point 1 again; train a LDA model and evaluate the model
with the same metrics

In [39]:
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)
y_predicted_train = lda_model.predict(X_train)
y_predicted_test = lda_model.predict(X_test)

In [40]:
get_model_metrics(y_train, y_predicted_train)

Accuracy: 0.86
Recall:  0.81
Precision:  0.88


In [41]:
get_model_metrics(y_test, y_predicted_test)

Accuracy: 0.82
Recall:  0.71
Precision:  0.88


In [42]:
lda_model.coef_.T

array([[ 0.00775993],
       [ 1.07450737],
       [ 0.79059315],
       [ 0.03740164],
       [ 0.00312264],
       [-0.76850344],
       [ 0.14315173],
       [-0.02352133],
       [ 1.56180146],
       [ 0.35808239],
       [ 0.48666282],
       [ 0.9030819 ],
       [ 0.49112669]])

In [43]:
lda_model.intercept_

array([-10.99519506])